In [1]:
from collections import defaultdict
from itertools import combinations
from time import time
import igraph as ig
import numpy as np
import random
import heapq

In [2]:
n = 10000
m = int(2*n)

def GNM(n, m):
    forbiddenEdges = defaultdict(set)
    rangeSet = set(range(n))    
    edges = []
    for _ in range(m):
        i = random.randrange(n)
        j = random.choice(tuple(
            rangeSet.difference(forbiddenEdges[i].union({i}))
        ))        
        forbiddenEdges[i].add(j)
        forbiddenEdges[j].add(i)
        edges.append((i,j))    
    g = ig.Graph(n) 
    g.add_edges(edges)
    return g

start = time()
g = GNM(n, m)
print(f"n:{n}, m: {m}, time: {round(time()-start, 4)} s")
#g.save(f"GNMn{n}m{m}.net")
#ig.plot(g)

n:10000, m: 20000, time: 10.2656 s


In [3]:
n = 10000
p = 0.1

def GNP(n, p):       
    possibleEdges = combinations(range(n), 2)
    r = random.random    
    g = ig.Graph(n)
    g.add_edges(edge for edge in possibleEdges if r() < p)
    return g

start = time()
g = GNP(n, p)
print(f"n:{n}, p: {p}, time: {round(time()-start, 4)} s")
#g.save(f"GNPn{n}p{str(p).replace('.', '_')}.net")
#ig.plot(g)

n:10000, p: 0.1, time: 7.9285 s


In [4]:
n = 10000
p = 0.75

def WS(n, p):
    g = ig.Graph(n)
    sources = list(range(n))
    forbiddenEdges = {i: {i} for i in sources}
    for i in (1, 2):
        targets = sources[i:] + sources[0:i]
        g.add_edges(list(zip(sources, targets)))
        for source, target in zip(sources, targets):
            forbiddenEdges[source].add(target)
            forbiddenEdges[target].add(source)
    sources = set(sources)
    toDelete = []
    toAdd = []
    for old_i, old_j in g.get_edgelist():
        if random.random() < p:
            forbiddenEdges[old_i].remove(old_j)
            forbiddenEdges[old_j].remove(old_i)
            toDelete.append((old_i, old_j))
            
            new_i = random.randrange(n)
            new_j = random.choice(tuple(
                sources.difference(forbiddenEdges[new_i])
            ))
            forbiddenEdges[new_i].add(new_j)
            forbiddenEdges[new_j].add(new_i)
            toAdd.append((new_i, new_j))            

    g.delete_edges(toDelete)
    g.add_edges(toAdd)
    return g

start = time()
g = WS(n, p)
print(f"n:{n}, p: {p}, time: {round(time()-start, 4)} s")
#g.save(f"WSn{n}p{str(p).replace('.', '_')}.net")
#ig.plot(g)

n:10000, p: 0.75, time: 3.2512 s


In [10]:
n = 10000
m0 = 5
m = 5

def BA(n, m, m0):
    vertices = list(range(m0))
    forbiddenEdges = set()
    degrees = [m0-1] * m0    
    edges = []    
    for index in range(m0, n):
        forbiddenEdges.clear()      
        count = 0
        while count < m:
            target = random.choices(vertices, degrees)[0]
            if target not in forbiddenEdges:
                edges.append((index, target))
                forbiddenEdges.add(target)
                degrees[target] += 1                
                count += 1
        vertices.append(index)
        degrees.append(m)
    
    g = ig.Graph.Full(n=m0)
    g.add_vertices(n-m0)
    g.add_edges(edges)
    return g

start = time()
g = BA(n, m, m0)
print(f"n:{n}, m: {m}, m0: {m0}, time: {round(time()-start, 4)} s")
#g.save(f"BAn{n}m{m}M{m0}.net")
#ig.plot(g)

n:10000, m: 5, m0: 5, time: 18.8221 s


In [5]:
n = 10000

def CM(n):
    forbiddenEdges = defaultdict(set)
    rangeSet = set(range(n))
    g = ig.Graph(n)
    degrees = [1]
    edges = []

    while sum(degrees) % 2 != 0:
        degrees = np.random.randint(1, 5, n)
    stubs = dict(enumerate(degrees))    
    heap = [(-val, i) for i, val in enumerate(degrees)]
    heapq.heapify(heap)
    
    while stubs:     
        source = None
        while source not in stubs:
            v, source = heapq.heappop(heap)
        targets = rangeSet.difference(forbiddenEdges[source].union({source}))
        target = random.choice(tuple(targets))        
        
        forbiddenEdges[source].add(target)
        forbiddenEdges[target].add(source)
        edges.append((source, target))

        for idx in (source, target):
            stubs[idx] -= 1
            if stubs[idx] == 0:
                rangeSet.remove(idx)
                del stubs[idx]
        
        if source in stubs:
            heapq.heappush(heap, (-stubs[source], source))
        
    g.add_edges(edges)
    return g

start = time()
g = CM(n)
print(f"n:{n}, time: {round(time()-start, 4)} s")
#g.save(f"CMn{n}.net")
#ig.plot(g)

n:10000, time: 10.5036 s
